# 装饰器

### deco 返回inner函数对象

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [2]:
@deco
def target():
    print('running target()')

In [3]:
target()
target

running inner()


<function __main__.deco.<locals>.inner()>

### 装饰器在定义时就已运行

In [4]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x7f0c4c511e18>)
running register(<function f2 at 0x7f0c4c4b5268>)


In [5]:
main()

running main()
registry -> [<function f1 at 0x7f0c4c511e18>, <function f2 at 0x7f0c4c4b5268>]
running f1()
running f2()
running f3()


In [6]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [7]:
from dis import dis
dis(f1)

 10           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('running f1()')
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


### 闭包/closure

In [8]:
class Averager():
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
def make_average():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [9]:
avg = Averager()
print(avg(10), avg(11), avg(12))

10.0 10.5 11.0


In [10]:
avg = make_average()
print(avg(10), avg(11), avg(12))

10.0 10.5 11.0


in averager function, series is a free variable, 指未在本地作用域中绑定的变量。averager的闭包延伸到函数的作用域之外，包含自由变量series的绑定

In [11]:
print(avg.__code__.co_varnames, avg.__code__.co_freevars)

('new_value', 'total') ('series',)


In [12]:
avg.__closure__

(<cell at 0x7f0c4c5458b8: list object at 0x7f0c4c4ae908>,)

In [13]:
avg.__closure__[0].cell_contents

[10, 11, 12]

闭包是一种函数，它会保留定义含函数时存在的自由变量的绑定。值得注意的是，只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量。

### 使用nonlocal

下面是更高效率的版本：只存储目前的sum和count，但存在问题。

In [14]:
def make_averager2():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / conut
    return averager

In [15]:
avg = make_averager2()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

这是因为count和total是不可变类型，`count += 1`的作用其实与`count = count + 1`一样，我们在averager的定义体中为count赋值了，这会把count变为局部变量。而在`make_average`中我们只是调用了`series.append`。

对于数字、字符串等不可变类型来说，只能读取不能更新，否则会隐式创建局部变量，就不能保存在闭包中了。

对此问题，Python3中引入了`nonlocal`声明，用于把变量标记为自由变量。

In [16]:
def make_averager3():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [17]:
avg = make_averager3()
avg(10)

10.0

### Decorator

输出函数的运行时间

In [72]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter() # perf_counter: Return the value (in fractional seconds) of a performance counter,
                                 # substitute for time.clock()
        result = func(*args) # func is a free variable
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [73]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [74]:
print('*'*40, "Calling snooze(.123)")
snooze(.123)
print('*'*40, "Calling factorial(6)")
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12322570s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000140s] factorial(1) -> 1
[0.00002090s] factorial(2) -> 2
[0.00005030s] factorial(3) -> 6
[0.00068320s] factorial(4) -> 24
[0.00072960s] factorial(5) -> 120
[0.00074970s] factorial(6) -> 720
6! = 720


由于`@clock`在定义函数时就已经运行了，所以factorial保存的其实是clocked函数的引用：

In [75]:
factorial.__name__

'clocked'

这就是装饰器的典型行为：把被装饰的函数替换成新函数，且二者接受相同的参数，且（通常）返回被装饰的函数本该返回的值，同时进行一些额外的操作。

上述装饰器有一些缺陷：不支持关键字参数，并且遮盖了原有函数的__name__和__doc__属性。下例使用`functools.wraps`装饰器把相关属性从func中复制到clocked中，且能正常处理关键字参数。

In [76]:
import time
import functools

def clock2(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [77]:
@clock2
def snooze(seconds):
    time.sleep(seconds)

@clock2
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*'*40, "Calling snooze(.123)")
snooze(.123)
print('*'*40, "Calling factorial(6)")
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12398815s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000143s] factorial(1) -> 1
[0.00002241s] factorial(2) -> 2
[0.00003576s] factorial(3) -> 6
[0.00004888s] factorial(4) -> 24
[0.00081325s] factorial(5) -> 120
[0.00087023s] factorial(6) -> 720
6! = 720


In [78]:
factorial.__name__

'factorial'

### 标准库中的装饰器

`functools.lru_cache` 实现memoization功能，保存“Least Recently Used”的结果。

In [91]:
@clock2
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [92]:
print(fibonacci(10))

[0.00000191s] fibonacci(0) -> 0
[0.00000167s] fibonacci(1) -> 1
[0.00021434s] fibonacci(2) -> 1
[0.00000167s] fibonacci(1) -> 1
[0.00000167s] fibonacci(0) -> 0
[0.00000167s] fibonacci(1) -> 1
[0.00007319s] fibonacci(2) -> 1
[0.00014591s] fibonacci(3) -> 2
[0.00044775s] fibonacci(4) -> 3
[0.00000119s] fibonacci(1) -> 1
[0.00000143s] fibonacci(0) -> 0
[0.00000167s] fibonacci(1) -> 1
[0.00007224s] fibonacci(2) -> 1
[0.00014234s] fibonacci(3) -> 2
[0.00000119s] fibonacci(0) -> 0
[0.00000191s] fibonacci(1) -> 1
[0.00007224s] fibonacci(2) -> 1
[0.00000119s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00000167s] fibonacci(1) -> 1
[0.00007272s] fibonacci(2) -> 1
[0.00014377s] fibonacci(3) -> 2
[0.00039649s] fibonacci(4) -> 3
[0.00060844s] fibonacci(5) -> 5
[0.00112677s] fibonacci(6) -> 8
[0.00000119s] fibonacci(1) -> 1
[0.00000143s] fibonacci(0) -> 0
[0.00000310s] fibonacci(1) -> 1
[0.00007176s] fibonacci(2) -> 1
[0.00013685s] fibonacci(3) -> 2
[0.00000119s] fibonacci(0) -> 0
[0.00000

In [95]:
import functools

# Call lru_cache as a normal function with '()' operator, as lru_cache can accept configuration parameters.
# Parameters of lru_cache:
#     Param maxsize : default: 128, how many results to cache.
#     Param typed : default: False, whether to save different types of parameters seperately.
@functools.lru_cache() 
@clock2
def fibonacci2(n):
    if n < 2:
        return n
    return fibonacci2(n-2) + fibonacci2(n-1)

lru_cache使用字典来存储结果，而且键根据调用时传入的定位参数和关键字参数创建，所以被其装饰的函数的所有参数都必须hashable

In [96]:
print(fibonacci2(10))

[0.00000238s] fibonacci2(0) -> 0
[0.00000215s] fibonacci2(1) -> 1
[0.00021291s] fibonacci2(2) -> 1
[0.00000286s] fibonacci2(3) -> 2
[0.00028872s] fibonacci2(4) -> 3
[0.00000310s] fibonacci2(5) -> 5
[0.00036550s] fibonacci2(6) -> 8
[0.00000262s] fibonacci2(7) -> 13
[0.00043178s] fibonacci2(8) -> 21
[0.00000262s] fibonacci2(9) -> 34
[0.00049877s] fibonacci2(10) -> 55
55
